In [7]:

!pip install nltk tensorflow


In [8]:
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import json
import random


In [9]:
{
  "intents": [
    {"tag": "greeting",
     "patterns": ["Hi", "Hello", "Is anyone there?", "Hey", "Hola", "Bonjour", "Namaste"],
     "responses": ["Hello!", "Hi there!", "How can I help you?", "Hi! How can I assist you?", "Hello, how can I help?"],
     "context_set": ""
    },

  ]
}

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hello',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Bonjour',
    'Namaste'],
   'responses': ['Hello!',
    'Hi there!',
    'How can I help you?',
    'Hi! How can I assist you?',
    'Hello, how can I help?'],
   'context_set': ''}]}

In [10]:
nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

# Process intents JSON file
with open('intents.json') as file:
    data = json.load(file)

words = []
classes = []
documents = []
ignore_words = ['?', '!']

# Loop through each sentence in the intent patterns
for intent in data['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # Add documents in the corpus
        documents.append((w, intent['tag']))
        # Add to classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# Sort classes
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


113 documents
30 classes ['Identity', 'activity', 'age', 'appreciate', 'contact', 'covid19', 'cricket', 'datetime', 'exclaim', 'goodbye', 'google', 'greeting', 'greetreply', 'haha', 'inspire', 'insult', 'jokes', 'karan', 'news', 'nicetty', 'no', 'options', 'programmer', 'riddle', 'song', 'suggest', 'thanks', 'timer', 'weather', 'whatsup']
130 unique lemmatized words ["'m", "'s", ',', '10', '19', 'a', 'age', 'am', 'anyone', 'are', 'ask', 'awesome', 'bad', 'bbye', 'be', 'best', 'bye', 'can', 'contact', 'could', 'covid', 'creator', 'cricket', 'current', 'date', 'day', 'designed', 'developer', 'do', 'doing', 'dumb', 'fine', 'for', 'funny', 'get', 'good', 'goodbye', 'google', 'great', 'haha', 'he', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'hot', 'how', 'i', 'idiot', 'india', 'inspiration', 'inspires', 'internet', 'is', 'it', 'joke', 'karan', 'know', 'later', 'latest', 'laugh', 'lmao', 'lol', 'lost', 'made', 'make', 'malik', 'match', 'me', 'motivates', 'namaste', 'news', '

In [11]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
# Convert the sublists to NumPy arrays before creating the

In [12]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

print("Training data created")


Training data created


In [13]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
history = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)


Epoch 1/200
23/23 [==============================] - 1s 3ms/step - loss: 3.4366 - accuracy: 0.0177
Epoch 2/200
23/23 [==============================] - 0s 2ms/step - loss: 3.3854 - accuracy: 0.0442
Epoch 3/200
23/23 [==============================] - 0s 2ms/step - loss: 3.3260 - accuracy: 0.0708
Epoch 4/200
23/23 [==============================] - 0s 2ms/step - loss: 3.2924 - accuracy: 0.0973
Epoch 5/200
23/23 [==============================] - 0s 2ms/step - loss: 3.2366 - accuracy: 0.1504
Epoch 6/200
23/23 [==============================] - 0s 3ms/step - loss: 3.2051 - accuracy: 0.1239
Epoch 7/200
23/23 [==============================] - 0s 2ms/step - loss: 3.1345 - accuracy: 0.1504
Epoch 8/200
23/23 [==============================] - 0s 3ms/step - loss: 3.0558 - accuracy: 0.1770
Epoch 9/200
23/23 [==============================] - 0s 3ms/step - loss: 3.0433 - accuracy: 0.2035
Epoch 10/200
23/23 [==============================] - 0s 3ms/step - loss: 2.9499 - accuracy: 0.1504
Epoch 11/

In [14]:
model.save('chatbot_model.h5', history)
print("Model saved")


Model saved


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result


In [ ]:
# Load trained model
model = tf.keras.models.load_model('chatbot_model.h5')

# Chatbot loop
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break

    # Get intent predictions
    intents = predict_class(user_input, model)

    # Choose a random response from intents
    bot_response = get_response(intents, data)

    print("Bot:", bot_response)


You: hi
1/1 [==============================] - 0s 88ms/step
Bot: Good to see you again
You: thank you
1/1 [==============================] - 0s 21ms/step
Bot: My pleasure
You: hello
1/1 [==============================] - 0s 20ms/step
Bot: Hi there, how can I help?
You: please tell me city name
1/1 [==============================] - 0s 33ms/step
Bot: The world tongue-twister champion just got arrested. I hear they're gonna give him a really tough sentence.
You: hello
1/1 [==============================] - 0s 22ms/step
Bot: Good to see you again
